## Advise re identity architecture

Initialize

In [2]:
#r "nuget: Microsoft.SemanticKernel, 1.0.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.0.1-alpha"

#r "nuget: System.Linq.Async, 6.0.1"

#!import ../config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Connectors.OpenAI;

using Kernel = Microsoft.SemanticKernel.Kernel;

var (useAzureOpenAI, model, azureEndpoint, apiKey, bingApiKey, orgId) = Settings.LoadFromFile();
var kernel = Microsoft.SemanticKernel.Kernel.CreateBuilder()
            .AddAzureOpenAIChatCompletion(
                model,   // deployment name
                azureEndpoint, // Azure OpenAI Endpoint
                apiKey)      // Azure OpenAI Key
            .Build();
#pragma warning disable SKEXP0011, SKEXP0003, SKEXP0052
var memoryBuilder = new MemoryBuilder();
memoryBuilder.WithAzureOpenAITextEmbeddingGeneration(
    "mrtextembeddingada002",
    azureEndpoint, 
    apiKey)
.WithMemoryStore(new VolatileMemoryStore());   
var memory = memoryBuilder.Build();            

Installed Packages Microsoft.SemanticKernel, 1.0.1 Microsoft.SemanticKernel.Plugins.Memory, 1.0.1-alpha System.Linq.Async, 6.0.1

Define/load plugin to facilitate use of chat history

In [18]:
using Microsoft.SemanticKernel.Plugins.Memory;

kernel.Plugins.Clear();

#pragma warning disable SKEXP0052

// TextMemoryPlugin provides the "recall" function
kernel.ImportPluginFromObject(new TextMemoryPlugin(memory));
var appFunction = kernel.CreateFunctionFromPrompt(File.ReadAllText("./ApplicationDiscovery.txt"), new OpenAIPromptExecutionSettings { MaxTokens = 200, Temperature = 0.8 });
var userFunction = kernel.CreateFunctionFromPrompt(File.ReadAllText("./UserIdentity.txt"), new OpenAIPromptExecutionSettings { MaxTokens = 200, Temperature = 0.8 });

Define arguments

In [19]:
#pragma warning disable SKEXP0052
var arguments = new KernelArguments();
arguments[TextMemoryPlugin.CollectionParam] = "history";
arguments[TextMemoryPlugin.LimitParam] = "2";
arguments[TextMemoryPlugin.RelevanceParam] = "0.8";

Define chat item

In [20]:
var history = "";
arguments["history"] = history;
Func<string, Task> Chat = async (string input) => {
    // Save new message in the kernel arguments
    arguments["userInput"] = input;
    var answer = await userFunction.InvokeAsync(kernel, arguments);
    var result = $"\nUser: {input}\nassistant: {answer}\n";
    history += result;
    arguments["history"] = history;
    Console.WriteLine(result);
};

Define chat loop

In [21]:
using System.Globalization;
using Microsoft.DotNet.Interactive;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;
do
{
    var user = await InteractiveKernel.GetInputAsync("Please enter your question or response");
    if (String.IsNullOrEmpty(user) || user.StartsWith("q", true, CultureInfo.CurrentCulture)) break;
    await Chat(user);
} while (true);
Console.WriteLine("Finished.");


User: I have a console app for my employees
assistant: Based on the given information, it seems like the correct choice for authenticating and authorizing users of the application would be Azure B2B. This option allows for access and authorization to be controlled by the company that owns the application, while also allowing access for employees of other companies.


User: But I want other companies to manage their own authorization
assistant: Description: This application is used by employees of our company as well as employees from other companies, but all access and authorization is controlled by our company.
Answer: Use B2B authentication and authorization.


User: No, other companies will manage their own authorization
assistant: Description: This application is used by employees of our company as well as employees of other companies, but all access and authorization is controlled by our company.
Answer: Use Entra ID B2B solution.

Finished.


Some tests

In [13]:
await Chat("Here is my system description: it consists of a single web application showing html content.");


User: Here is my system description: it consists of a single web application showing html content.
ChatBot: Got it. So your system consists of a single web application that shows HTML content. Is that correct?

